# 推薦系統的模型驗證

## 排序指標

### MRR

In [10]:
from typing import List

import numpy as np
import pandas as pd

In [11]:
def rr_at_k(one_user_relevances: List[int], k: int) -> float:
    """ Reciprocal Rank at K """
    for rank, relevance in enumerate(one_user_relevances[:k], start=1):
        # 正樣本
        if relevance == 1:
            return 1.0 / rank
    # 未命中正樣本
    return 0.0


def mrr_at_k(all_user_relevances: List[List[int]], k: int) -> float:
    """ Mean Reciprocal Rank at K """
    return np.mean([rr_at_k(one_user_relevances, k) for one_user_relevances in all_user_relevances])

In [12]:
## 測試樣本

one_user_relevances = [0, 0, 1, 0, 1]
k = 5
print(f"RR@{k}: {rr_at_k(one_user_relevances, k)}")

all_user_relevances = [
    [0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
]
for user_relevances in all_user_relevances:
    print(f"User relevances: {user_relevances} -> RR@{k}: {rr_at_k(user_relevances, k)}")
print(f"MRR@{k}: {mrr_at_k(all_user_relevances=all_user_relevances, k=k)}")

RR@5: 0.3333333333333333
User relevances: [0, 0, 1, 0, 1] -> RR@5: 0.3333333333333333
User relevances: [1, 0, 0, 0, 0] -> RR@5: 1.0
User relevances: [0, 0, 0, 1, 0] -> RR@5: 0.25
MRR@5: 0.5277777777777778
